In [62]:
"""
Project for Olin College, Data Science, Spring 2017.

Data from:
https://www.nass.usda.gov/Surveys/Guide_to_NASS_Surveys/Bee_and_Honey/
http://usda.mannlib.cornell.edu/MannUsda/viewDocumentInfo.do?documentID=1191


unzip the csv zip files with

$  for VARIABLE in 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016; 
$  do   unzip *$VARIABLE.zip -d $VARIABLE; 
$  done

or pull them from my git.
"""

import numpy as np
import pandas
import os
import string

In [63]:
def get_all_csvs(csv_years=[2002+i for i in range(15)], csv_path="data/csvs/"):
    '''
    Returns a list of all csv files in the (hard-coded) year-directory combinations.
    '''
    all_csvs = []
    for i in csv_years:
        this_path = csv_path + str(i) + "/"
        these_files = os.listdir(this_path)
        these_csvs = [(this_path + i) for i in these_files if i[-4:] == ".csv"]
        all_csvs.extend(these_csvs)
    return all_csvs

In [64]:
class Csv_obj(object):
    def __init__(self, filename):
        self.name = filename
        year_start = filename.find("20")
        self.year = int(filename[year_start:year_start+4])
        self.data_type = filename[(filename.find("_")) : (filename.find(".csv"))]
        self.df = sanitize_then_load_csv(filename)
        
    def __str__(self):
        return str(self.year) + "\t" + self.data_type
    
    def isnull(self):
        if self.df is None:
            return True
        else:
            return False

In [68]:
def sanitize_then_load_csv(filename, min_col_length=9, max_col_length = None):
    """
    Gets around the metadata-y columns by scanning for the length of .split(',').
    TODO: make it ignore the "format"/grammatical commas which occur within a single entry in the csv.
    """
    f = open(filename)
    lines_to_skip = []
    for i, line in enumerate(f.readlines()):
        l_split = line.split(',') # will also split internal commas in strings. todo: fix. low priority.
        if len(l_split) < min_col_length:
            lines_to_skip.append(i)
        elif max_col_length != None:
            if len(l_split) > max_col_length:
                lines_to_skip.append(i)
    try:
        df = pandas.read_csv(filename, skiprows = lines_to_skip)
    except Exception as err:
        print "Cannot read: ", filename, "\t ", str(err)[str(err).find("Expected"):]
        df = None
    return df

In [69]:
all_csvs = get_all_csvs()

In [70]:
csv_objs = [Csv_obj(i) for i in all_csvs]

Cannot read:  data/csvs/2002/hony_all.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2002/hony_001.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2002/hony_003.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2002/hony_002.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2003/hony_all.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2003/hony_001.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2003/hony_003.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2003/hony_002.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2004/hony_all.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2004/hony_001.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2004/hony_003.csv 	  global name 'target_file' is not defined
Cannot read:  data/csvs/2004/hony_002.csv 	  global na